# Techniques d'anonymisation

 <font color='red'>1- Maskage de valeurs d'attribut</font> 


***
Analysons les periodes de livraison selon l’adresse et le nombre de colis;
En considérant le fichier data_livraison.csv, procédons en suite au maskage de l’attribut code_postal. Le
format en sortie serait de la forme : ZY* *** où ZY sont les deux premiers digits du code postal
original.
***

In [50]:
import pandas as pd
df = pd.read_csv("data_livraison.csv")


* <font color='green'> Informations des clients avant l'opération de maskage de leurs code postal</font> 

In [53]:
df

,code postal,periode livraison,nombre colis
0,G1X 3S7,10H-12H,2
1,G2H5 5K3,27F-21H,4
2,G1W 5K3,19H-28X,1


 * <font color='green'> Résulat aprés maskage du code postal </font> 

* masker le code postal sur <font color='red'>2 digits</font> 


In [54]:
df["code"] = df["code postal"].map(lambda x: x[0:2] +"* ***")
df

,code postal,periode livraison,nombre colis,code
0,G1X 3S7,10H-12H,2,G1* ***
1,G2H5 5K3,27F-21H,4,G2* ***
2,G1W 5K3,19H-28X,1,G1* ***


<font color='red'> 2- Utilisation de pseudonyme </font> 

***
Procédons à la suppression de l’attribut PATIENT par la recherche  de l’effet du reactif, en termes du résultat
ou score sur les patients dans le jeu données le fichier data_patient.csv
***

<font color='red'> 2-1  Utilisation de pseudonyme à 1 niveau </font> 

In [55]:
import pandas as pd
df = pd.read_csv("reactif_patient.csv")


* <font color='green'> Informations des patients avant l'opération pseudonyme (sur les noms des patients) à 1 niveau</font> 

In [56]:
df

,patient,reactif,resultat
0,Fabrice Laprise,A,79
1,Eric Cloutier,A,81
2,Annie Houle,B,47


In [57]:
#Déclarer la table de correspondance
d = {'patient':['Fabrice Laprise','Eric Cloutier','Annie Houle'],'pseudonyme':[6825,4514,6728]}
table_cor = pd.DataFrame(d)


* <font color='green'> Les pseudonyme des patients aprés l'opération de pseudonyme (sur les noms des patients) à 1 niveau</font> 

In [58]:
table_cor

,patient,pseudonyme
0,Fabrice Laprise,6825
1,Eric Cloutier,4514
2,Annie Houle,6728


In [9]:
#anonymiser en effectuant un merge
res = pd.merge(df, table_cor)
res

,patient,reactif,resultat,pseudonyme
0,Fabrice Laprise,A,79,6825
1,Eric Cloutier,A,81,4514
2,Annie Houle,B,47,6728


In [10]:
#anonymiser en effectuant un merge avec une suppression de la serie patient
res = pd.merge(df, table_cor).drop('patient', axis=1)
res

,reactif,resultat,pseudonyme
0,A,79,6825
1,A,81,4514
2,B,47,6728


In [11]:
#renommer la serie pseudonyme vers patient
res = res.rename(columns={'pseudonyme': 'patient'})
res

,reactif,resultat,patient
0,A,79,6825
1,A,81,4514
2,B,47,6728


In [12]:
# Réarranger les colonnes en mettant patient en premier
res = res[['patient', 'reactif', 'resultat']]
res

,patient,reactif,resultat
0,6825,A,79
1,4514,A,81
2,6728,B,47


In [13]:
#sauvegarder le data transformé dqns un fichier de type csv
res.to_csv("reactif_patient_anonyme.csv", index=False)

<font color='red'> 2-2 Utilisation de pseudonyme à 2 niveaux </font> 

***
En considérant le fichier reactif_patient.csv, procédons à la pseudonymisation de Patient. Selon une table
de correspondance suivante :
***

| Patient | Pseudonyme |
| --- | --- |
| Fabrice Laprise | 6852 |
| Eric Cloutier | 4514 |
| Annie Houle | 6728 |

In [73]:
#Déclarer la table de pseudo lien
d = {'patient':['Fabrice Laprise','Eric Cloutier','Annie Houle'],
     'lien':["MN815","PS34M","XD48T"]}
table_cor = pd.DataFrame(d)
table_cor = pd.merge(table_cor,df)
table_cor

,patient,lien,reactif,resultat
0,Fabrice Laprise,MN815,A,79
1,Eric Cloutier,PS34M,A,81
2,Annie Houle,XD48T,B,47


In [74]:
#Déclarer la table de pseudo
d = {'lien':["MN815","PS34M","XD48T"],'pseudonyme':[6825,4514,6728]}
table_pseudo = pd.DataFrame(d)

* <font color='green'> Les informations des patients aprés l'opération de pseudonyme (sur les noms des patients) à 2 niveau</font>

In [75]:
table_pseudo

,lien,pseudonyme
0,MN815,6825
1,PS34M,4514
2,XD48T,6728


In [78]:
#Supposons qu'on a le dataframe complet patient-pseudo
table_patients = pd.merge(table_cor, table_pseudo)
table_patients

,patient,lien,reactif,resultat,pseudonyme
0,Fabrice Laprise,MN815,A,79,6825
1,Eric Cloutier,PS34M,A,81,4514
2,Annie Houle,XD48T,B,47,6728


In [79]:
#Obtenir le dataframe banque 
table_patients.drop(["reactif","resultat","pseudonyme"],axis=1, inplace=True)


In [80]:
table_patients

,patient,lien
0,Fabrice Laprise,MN815
1,Eric Cloutier,PS34M
2,Annie Houle,XD48T


In [37]:
#sauvegarder le data transformé
table_patients.to_csv("reactif_patient_banque.csv", index=False)
#Obtenir le dataframe entreprise
table_patients.drop("patient",axis=1, inplace=True)
table_patients

,lien
0,MN815
1,PS34M
2,XD48T


In [38]:
#sauvegarder le data transformé dqns un fichier de type csv
table_patients.to_csv("reactif_patient_entreprise.csv", index=False)


<font color='red'> 3- Generalisation </font> 

***
En considérant le fichier data_patient_age.csv, procédons à l'anonymisation de la série age en
utilisant les intervalles d’age au lieu de l’age exact. La table d’intervalle est la suivante :
***
| Age patient | Plage |
| --- | --- |
| Plage 1 | < 20 |
| Plage 2 | < 20 - 30 |
| Plage 3 | 31 - 40 |
| Plage 4 | 41 - 50 |
| Plage 5 | > 50 |

In [81]:
import pandas as pd
df = pd.read_csv("data_patient_age.csv")

* <font color='green'> Informations des patients avant de procéder a la généralisation (avec l'age exact du patient) </font> 

In [82]:
df

,patient,age,reactif,resultat
0,Fabrice Laprise,25,A,79
1,Eric Cloutier,35,A,81
2,Annie Houle,45,B,47


In [86]:
#appliquons une coorespondance avec categories ou plage des ages pour procéder au masquage l'age exacte du patient
df["age_interval"] = df["age"].map(lambda x: "<20" if x < 20 else
                                  "20-30" if x < 30 else
                                  "31-40" if x < 40 else
                                  "41-50" if x < 50 else
                                  ">50")
df.drop("age", axis=1, inplace=True)
df=df[["patient","age_interval", "reactif","resultat"]]
df

,patient,age_interval,reactif,resultat
0,Fabrice Laprise,20-30,A,79
1,Eric Cloutier,31-40,A,81
2,Annie Houle,41-50,B,47


In [87]:
df=df[["patient","age_interval", "reactif"]]

* <font color='green'> Informations des patients aprés l'application de la technique de généralisation (avec plage age)</font> 

In [88]:
df

,patient,age_interval,reactif
0,Fabrice Laprise,20-30,A
1,Eric Cloutier,31-40,A
2,Annie Houle,41-50,B
